In [1]:
import sys
sys.path.append("..")
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from utils.PDE_Net import DeepONet_NS, FNO2d, weight_init
from utils.DataGenerate_FNO import Dataset_FNO, Normalize, InNormalize
from torch.utils.tensorboard import SummaryWriter
from utils.utilities3 import *

torch.set_default_dtype(torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'utils'

In [ ]:
sys.path

In [ ]:
modes = 12
width = 100
dx = 0.5
n_x = int(8/dx)
n_y = int(3/dx)
p_x = int(800/n_x+1)
p_y = int(300/n_y+1)
self_split = 2

epochs = 1000
step_size = 50

batch_size = 300
learning_rate = 0.001
iterations = 2*epochs
max_norm = 5

n_train = 300
n_test = 100

path_trained_model = r'FNO/trained_model'

path_distance = r'train_data/distance'
path_label = r'train_data/label'

path_distance_test_name = r'distance_test.npy'
path_label_test_name = r'label_test.npy'
path_distance_test = r'test_data/distance'
path_label_test = r'test_data/label'

In [ ]:
train_distance = torch.empty(0, p_x, p_y, 3)
train_label = torch.empty(0, p_x, p_y, 1)
test_distance = torch.empty(0, p_x, p_y, 3)
test_label = torch.empty(0, p_x, p_y, 1)

file_list = os.listdir(path_distance)
for file in file_list:
	if file.endswith('.npy'):
			distance = np.load(os.path.join(path_distance, file))
			train_distance = torch.cat((train_distance, torch.Tensor(distance)), 0)

file_list = os.listdir(path_label)
for file in file_list:
	if file.endswith('.npy'):
			label = np.load(os.path.join(path_label, file))
			train_label = torch.cat((train_label, torch.Tensor(label)), 0)

file_list = os.listdir(path_distance_test)
for file in file_list:
	if file.endswith('.npy'):
			distance = np.load(os.path.join(path_distance_test, file))
			test_distance = torch.cat((test_distance, torch.Tensor(distance)), 0)

file_list = os.listdir(path_label_test)
for file in file_list:
	if file.endswith('.npy'):
			label = np.load(os.path.join(path_label_test, file))
			test_label = torch.cat((test_label, torch.Tensor(label)), 0)

# test_distance = np.load(os.path.join(path_distance_test, path_distance_test_name))
# test_label = np.load(os.path.join(path_label_test, path_label_test_name))

train_distance = torch.Tensor(train_distance)
train_label = torch.Tensor(train_label)
test_distance = torch.Tensor(test_distance)
test_label = torch.Tensor(test_label)

print(train_distance.shape)
print(train_label.shape)
print(test_distance.shape)
print(test_label.shape)

In [ ]:
x_normalizer = GaussianNormalizer(train_distance)
x_data = x_normalizer.encode(train_distance)
y_normalizer = GaussianNormalizer(train_label)
y_data = y_normalizer.encode(train_label)

x_normalizer_test = GaussianNormalizer(test_distance)
x_test = x_normalizer_test.encode(test_distance)
y_normalizer_test = GaussianNormalizer(test_label)
y_test = y_normalizer_test.encode(test_label)

In [3]:
model = FNO2d(modes,modes,width)
model.apply(weight_init)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
myloss = nn.MSELoss()

writer = SummaryWriter(r'runs\transtant_model')
begin_epoch = 0

unkown layer: SpectralConv2d()
unkown layer: SpectralConv2d()
unkown layer: SpectralConv2d()
unkown layer: SpectralConv2d()
unkown layer: Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
unkown layer: Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
unkown layer: MLP(
  (mlp1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (mlp2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
)
unkown layer: Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
unkown layer: Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
unkown layer: MLP(
  (mlp1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (mlp2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
)
unkown layer: Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
unkown layer: Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
unkown layer: MLP(
  (mlp1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  (mlp2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
)
unkown layer: Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
unkow

In [ ]:
file_list = os.listdir(path_trained_model)
if len(file_list) > 0:
	last_model = os.listdir(path_trained_model)[-1]
# save_path = os.path.join(path, 'model_{}.pth'.format(begin_epoch))
load_path = os.path.join(path_trained_model, last_model)
begin_epoch = load_model(load_path, optimizer, model)
model.to(device)

In [3]:
train_loader = DataLoader(torch.utils.data.TensorDataset(x_data, y_data), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

x, y = next(iter(train_loader))
print(torch.max(x))
print(torch.max(y))

In [81]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.5)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=10, verbose=True, min_lr=1e-6)
# epochs = 50000
for epoch in range(begin_epoch,epochs+1):
	model.train()
	train_loss_epoch = 0
	test_loss_epoch = 0
	for batch in train_loader:

		loss_train = 0
		data_x,data_y = batch

		x = data_x.float().to(device)			# [length,time_step,51,51,3]
		y = data_y.float().to(device)			# [length,time_step,51,51,3]

		pred = model(x)												# x:[length,51,51,3], pred:[length,51,51,3]
		
		pred = y_normalizer.decode(pred)
		y = y_normalizer.decode(y)
		
		loss_train = myloss(pred.clone(), y.clone())
			
		optimizer.zero_grad()
		train_loss_epoch = train_loss_epoch + loss_train.item()
		loss_train.backward(retain_graph=True)
		torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
		optimizer.step()
		# print('epoch:', epoch, 'loss_train:', loss_train.item())
	train_loss_epoch = train_loss_epoch / len(train_loader)
	print('epoch:', epoch, 'loss_train:', train_loss_epoch)
	scheduler.step(train_loss_epoch)
	
	model.eval()
	with torch.no_grad():
		for batch_test in test_loader:

			loss_test = 0
			test_x,test_y = batch_test
			
			test_x = test_x.float().to(device)			# [length,time_step,51,51,3]
			test_y = test_y.float().to(device)			# [length,time_step,51,51,3]

			pred_test = model(test_x)

			pred_test = y_normalizer_test.decode(pred_test)
			test_y = y_normalizer_test.decode(test_y)

			loss_test = myloss(pred_test.clone(), test_y.clone())

			test_loss_epoch = test_loss_epoch + loss_test.item()
		test_loss_epoch = test_loss_epoch/len(test_loader)
		print('epoch:', epoch, 'loss_test:', test_loss_epoch)

		# print('epoch:', epoch, 'loss_test:', loss_test.item())
	
	if epoch % 1000 == 0:
		save_path = os.path.join(path_trained_model, 'FNO_{}.pth'.format(epoch))
		save_model(save_path, epoch, optimizer, model)

		writer.add_scalar('loss_train', loss_train.item(), epoch)
		writer.add_scalar('loss_test', loss_test.item(), epoch)

	